#### Beschreibung:
Aufbereitung der Daten.

#### Workflow:
Echtdaten des Projektes, das zurzeit in der Konfiguration gesetzt ist, werden aus der MongoDB (Datenbank) ausgelesen. Anschließend werden sie in diesem Notebook für die weitere Verwendung aufbereitet.

# Daten bereinigen

In [1]:
%run ../Setup.ipynb

import pandas as pd
from isac.database.connection import database_connector as connector
from isac.database.structure import column_meta_helper as structureMeta
from isac import configuration

### Extraktion

#### Dokumente einlesen

In [2]:
connection = connector.DatabaseConnector(name = configuration.project_database).connect()
collection = connection.getCollection(configuration.collections.cleaned)

#### Get Brief Overview

In [3]:
df = pd.DataFrame(list(collection.find()))

df.sort_values(by=['time'])
df = df.reset_index(drop=True)

# order = list(namedColumns.values())
order = configuration.order

df[order].describe()

radius = 0.005 # radius in m, for 5 mm

virtual_force = df[order]['torque_set_f11'] / radius
# print(virtual_force)
df['Virtuelle Kraft'] = virtual_force

### Cleaning

#### Basic Math For Calculation

__P = ω * M__

P = Leistung  
ω = Omega, Kreifrequenz (Rad/Sec.)  
M = Drehmoment = r * F // Kann zur Berechnung herangezogen werden
r = Radius des Fräskopfes = 5 mm = 0,005 m  
F = Kraft, die gesuchte Größe

__⇒ F = M / r__  

In [4]:
radius = 0.005 # radius in m, for 5 mm

virtual_force = df[order]['torque_set_f11'] / radius
    
df['Virtuelle Kraft'] = virtual_force
focusColumns = ['time', 'Virtuelle Kraft', 'spindle_speed_actual_f3', 'torque_set_f11', 'spindle_power_f9']

df[focusColumns].tail()

,time,Virtuelle Kraft,spindle_speed_actual_f3,torque_set_f11,spindle_power_f9
3747,14.988,-58.6,4456.844330,-0.293,-115.048558
3748,14.992,81.4,4455.986023,0.407,129.429627
3749,14.996,258.0,4455.356598,1.290,632.767066
3750,15.000,-13.0,4457.187653,-0.065,28.762139
3751,15.004,25.6,4456.443787,0.128,-57.524279


#### Resampling auf 2 Millisekunden
#### Lineare Interpolation

In [5]:
df['time'] = pd.to_timedelta(df['time'], unit="s")
df = df.set_index('time')
df = df.resample('2L')
df = df.interpolate(method='linear') # spline, polynomial, -order
df = df.reset_index()